In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline


In [3]:
from splinator.estimators import LinearSplineLogisticRegression, CDFSplineCalibrator

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss, log_loss
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score
from sklearn.isotonic import IsotonicRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from splinator.metrics import expected_calibration_error, spiegelhalters_z_statistic

In [4]:
# Remove redundant imports - already imported above

In [5]:
# Remove redundant import - already imported above

In [6]:
X, y = make_classification(n_samples=50000, n_features=10, n_informative=2, n_redundant=8, flip_y=0.15)

# split train, test for calibration
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42
)

In [7]:
clf = GradientBoostingClassifier()

clf.fit(X_train, y_train)

clf_pred_dev = clf.predict_proba(X_dev)[:,1]
clf_pred_test = clf.predict_proba(X_test)[:,1]

In [8]:
roc_auc_score(y_score=clf_pred_dev, y_true=y_dev)

0.893854084279575

In [9]:
# Fit all spline-based calibrators
print("Fitting LinearSplineLogisticRegression...")
lslr = LinearSplineLogisticRegression(
        n_knots=20, 
        monotonicity="none", 
        minimizer_options={'disp': False}, 
        method='SLSQP', 
        two_stage_fitting_initial_size=2000
    )
lslr.fit(clf_pred_dev.reshape(-1, 1), y_dev)
lslr_pred = lslr.predict_proba(clf_pred_test.reshape(-1, 1))[:, 1]

print("Fitting CDFSplineCalibrator...")
cdf_cal = CDFSplineCalibrator(num_knots=6)
cdf_cal.fit(clf_pred_dev.reshape(-1, 1), y_dev)
cdf_pred = cdf_cal.predict_proba(clf_pred_test.reshape(-1, 1))[:, 1]

Fitting LinearSplineLogisticRegression...


AttributeError: 'LinearSplineLogisticRegression' object has no attribute 'predict_proba'

In [ ]:
print("Fitting PyGAM...")
from pygam import GAM, s, te

gam_model = GAM(s(0, n_splines=20, spline_order=1), distribution='binomial', link='logit', fit_intercept=True)
gam_model.fit(clf_pred_dev.reshape(-1, 1), y_dev)
gam_model_pred = gam_model.predict_proba(clf_pred_test.reshape(-1, 1))

In [ ]:
# Already imported at the top

In [ ]:
# Create comparison dataframe
df = pd.DataFrame({
    'uncalibrated': clf_pred_test,
    'y_true': y_test,
    'lslr': lslr_pred,
    'cdf_spline': cdf_pred,
    'pygam': gam_model_pred,
})

In [ ]:
# Calculate metrics for all methods
methods = ['uncalibrated', 'lslr', 'cdf_spline', 'pygam']
results = []

for method in methods:
    brier = brier_score_loss(y_test, df[method])
    logloss = log_loss(y_test, df[method])
    ece = expected_calibration_error(y_test, df[method])
    z_stat = spiegelhalters_z_statistic(y_test, df[method])
    
    results.append({
        'Method': method,
        'Brier Score': brier,
        'Log Loss': logloss,
        'ECE': ece,
        'Spiegelhalter Z': z_stat
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

In [ ]:
# Plot calibration curves for all methods
plt.figure(figsize=(12, 8))

# Sample data for cleaner visualization
sample_size = 1000
sample_idx = np.random.choice(len(df), sample_size, replace=False)
sample_df = df.iloc[sample_idx].sort_values('uncalibrated')

# Plot each method
plt.plot(sample_df['uncalibrated'], sample_df['lslr'], 
         label='LinearSplineLogisticRegression', linewidth=2, alpha=0.8)
plt.plot(sample_df['uncalibrated'], sample_df['cdf_spline'], 
         label='CDFSplineCalibrator', linewidth=2, alpha=0.8)
plt.plot(sample_df['uncalibrated'], sample_df['pygam'], 
         label='PyGAM', linewidth=2, alpha=0.8)

# Add diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', label='Perfect calibration', alpha=0.5)

plt.xlabel('Uncalibrated Probability', fontsize=12)
plt.ylabel('Calibrated Probability', fontsize=12)
plt.title('Calibration Curves Comparison', fontsize=14)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()


In [ ]:
# Compare the spline transformations more closely
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Sort by uncalibrated scores for better visualization
sorted_idx = np.argsort(clf_pred_test)
x_sorted = clf_pred_test[sorted_idx]

# Plot each spline method's transformation
spline_methods = [
    (lslr_pred[sorted_idx], 'LinearSplineLogisticRegression', axes[0]),
    (cdf_pred[sorted_idx], 'CDFSplineCalibrator', axes[1]),
    (gam_model_pred[sorted_idx], 'PyGAM', axes[2])
]

for pred, title, ax in spline_methods:
    # Plot transformation
    ax.plot(x_sorted, pred, linewidth=2, label='Calibrated', color='blue')
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='No change')
    
    # Highlight regions where calibration changes most
    diff = np.abs(pred - x_sorted)
    ax.fill_between(x_sorted, x_sorted, pred, where=(diff > 0.05), 
                    alpha=0.3, color='red', label='Large adjustment (>0.05)')
    
    ax.set_xlabel('Uncalibrated Probability', fontsize=11)
    ax.set_ylabel('Calibrated Probability', fontsize=11)
    ax.set_title(title, fontsize=12)
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
# Create a summary comparison plot
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Normalize metrics for better visualization
metrics_normalized = results_df.copy()
metrics_normalized['Brier Score'] = 1 - metrics_normalized['Brier Score'] / metrics_normalized.loc[0, 'Brier Score']
metrics_normalized['Log Loss'] = 1 - metrics_normalized['Log Loss'] / metrics_normalized.loc[0, 'Log Loss']
metrics_normalized['ECE'] = 1 - metrics_normalized['ECE'] / metrics_normalized.loc[0, 'ECE']
metrics_normalized['Spiegelhalter Z'] = 1 - np.abs(metrics_normalized['Spiegelhalter Z']) / np.abs(metrics_normalized.loc[0, 'Spiegelhalter Z'])

# Plot bars
x = np.arange(len(methods))
width = 0.2

metrics_to_plot = ['Brier Score', 'Log Loss', 'ECE', 'Spiegelhalter Z']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for i, metric in enumerate(metrics_to_plot):
    offset = (i - 1.5) * width
    ax.bar(x + offset, metrics_normalized[metric], width, 
           label=f'{metric} improvement', color=colors[i], alpha=0.8)

ax.set_xlabel('Method', fontsize=12)
ax.set_ylabel('Relative Improvement (higher is better)', fontsize=12)
ax.set_title('Calibration Performance Comparison', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(methods, rotation=45, ha='right')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

# Print percentage improvements
print("\nPercentage improvements over uncalibrated:")
for i in range(1, len(results_df)):
    print(f"\n{results_df.loc[i, 'Method']}:")
    print(f"  Brier Score: {(1 - results_df.loc[i, 'Brier Score']/results_df.loc[0, 'Brier Score']) * 100:.1f}%")
    print(f"  Log Loss: {(1 - results_df.loc[i, 'Log Loss']/results_df.loc[0, 'Log Loss']) * 100:.1f}%")
    print(f"  ECE: {(1 - results_df.loc[i, 'ECE']/results_df.loc[0, 'ECE']) * 100:.1f}%")
    print(f"  |Spiegelhalter Z|: {(1 - abs(results_df.loc[i, 'Spiegelhalter Z'])/abs(results_df.loc[0, 'Spiegelhalter Z'])) * 100:.1f}%")
